[원본 코랩 주소](https://colab.research.google.com/drive/1nBaePtwcW_ds7OQdFebcxB91n_aORQY5)

## 실행하는 방법

- **중요**: 모바일에서 접속했다면 꼭 **데스크탑 보기**를 켤 것
1. 상단 메뉴 중 `파일` > `드라이브에 사본 저장`
1. `실행` 셀의 좌측에 있는 있는 *재생 아이콘* 클릭
1. 조금 기다리면 웹 UI 주소 출력됨 (`https://xxxxxx.gradio.app`)
  <br>처음 실행할 땐 이거 저거 받아와야해서 조금 오래 걸림...
  <br>**빨간 오류 메세지**만 안뜨면 됨

## 자주 하는 질문



- Q. `AssertionError: Torch is not able to use GPU...`
  <br>A. GPU 못 찾아서 발생하는 오류, 코랩이면 GPU 런타임이 아니여서 발생하는 오류
  <br>상단 `런타임` > `런타임 유형 변경` > `GPU`로 변경하고 재실행

- Q. `백엔드를 할당하지 못했습니다. GPU이(가) 있는 백엔드를 사용할 수 없습니다...`
  <br>구글 부계정 만들어서 로그인하고 시도하면 될거임
  <br>코랩 Pro랑 Pro+ 전부 종량제로 바뀐 뒤로 돈낭비니까
  <br>돈 내고 싶으면 가능한 [NovelAI](https://novelai.net/) 쓰거나 [vast.ai](https://vast.ai) 같은 싼 GPU 호스팅/렌탈 서비스 추천함

- Q. **뭔가 안됨... 오류 뜨는데 잘 몰?루겠음...**
  <br>A. [만든 놈 미니 갤러리](https://gall.dcinside.com/mini/board/lists/?id=owo) 또는 디스코드 `aeon#4285`

- Q. **똥컴도 가능함?**
  <br>A. 구글 서버 쓰는거고 결과만 보여주는거라 상관 없음

- Q. **코랩 밖에서도 실행할 수 있음?**
  <br>A. 테스트는 안해봤는데 아마 될거임... 구글 드라이브 연동만 제외하고

- Q. **NAI 유출 모델 돌아감?**
  <br>A. ㅇㅇ

## 참고 주소

- https://rentry.org/voldy
- https://rentry.org/sdmodels
- https://cyberes.github.io/stable-diffusion-models/
- https://github.com/AUTOMATIC1111/stable-diffusion-webui
- https://public.vmm.pw/aeon/models

### 국내 관련 커뮤니티

- [아카라이브 AI그림 채널](https://arca.live/b/aiart)
- [아카라이브 AI그림 채널 위키](https://arca.live/w/aiart/FrontPage)
- [디시인사이드 AI 창작 마이너 갤러리](https://gall.dcinside.com/m/aicreate)
- [디시인사이드 특이점이 온다 마이너 갤러리](https://gall.dcinside.com/m/thesingularity)

## 실행

In [ ]:
# @title 실행하기
# fmt: off
from genericpath import isfile
import os
import sys
import platform
import re
import glob
import json
from tkinter.messagebox import NO
import requests
from typing import Union, Callable, List
from importlib.util import find_spec
from pathlib import Path
from shutil import rmtree
from collections import Counter
from IPython.display import display
from ipywidgets import widgets

def format_styles(styles: dict) -> str:
    return ';'.join(map(lambda kv: ':'.join(kv), styles.items()))

html_dialog = widgets.HTML()
html_dialog_presets = {
    'default': {
        'display': 'inline-block',
        'padding': '.5em',
        'background-color': 'black',
        'font-size': '1.25em',
        'line-height': '1em',
        'color': 'white',
    },
    'error': {
        'border-left': '6px solid red'
    }
}

html_logger_styles = {
    'overflow-x': 'auto',
    'max-width': '700px',
    'padding': '1em',
    'background-color': 'black',
    'white-space': 'pre-wrap',
    'font-family': 'monospace',
    'font-size': '1em',
    'line-height': '1em',
    'color': 'white'
}
html_logger = widgets.HTML(
    value=f'<div id="logger" style="{format_styles(html_logger_styles)}">')
html_logger.raw = ''


def dialog(msg, preset:str=None, styles=html_dialog_presets['default']) -> None:
    if preset and preset in html_dialog_presets:
        styles = {
            **html_dialog_presets['default'],
            **html_dialog_presets[preset],
            **styles
        }

    html_dialog.value = f'<div style="{format_styles(styles)}">{msg}</div>'


def log(msg, newline=True, styles={}, bold=False) -> None:
    if bold:
        styles['font-weight'] = 'bold'

    if newline:
        msg += '\n'

    html_logger.raw += msg
    html_logger.value += f'<span style="{format_styles(styles)}">{msg}</span>'


# ==============================
# 서브 프로세스
# ==============================
running_subprocess = None


def execute_subprocess(args: Union[str, List[str]], parser: Callable=None,
                       throw=True, **kwargs) -> int:
    global running_subprocess

    # 이미 서브 프로세스가 존재한다면 예외 처리하기
    if running_subprocess:
        raise Exception('하위 프로세스가 실행되고 있습니다')

    if isinstance(args, str):
        log(f'=> {args}', styles={'color':'yellow'})
    else:
        log(f"=> {' '.join(args)}", styles={'color':'yellow'})

    html_logger.value += '<div style="padding-left:1em">'

    # 서브 프로세스 만들기
    from subprocess import Popen, PIPE, STDOUT
    running_subprocess = Popen(
        args,
        stdout=PIPE,
        stderr=STDOUT,
        encoding='utf-8',
        **kwargs,
    )

    running_subprocess.output = ''

    # 프로세스 출력 위젯에 리다이렉션하기
    while running_subprocess.poll() is None:
        # 출력이 비어있다면 넘어가기
        out = running_subprocess.stdout.readline()
        if not out:
            continue

        # 프로세스 출력 버퍼에 추가하기
        running_subprocess.output += out

        # 파서가 없거나 또는 파서 실행 후 반환 값이 거짓이라면 로깅하기
        if not parser or not parser(out):
            log(out, newline=False, styles={'color': '#AAA'})

    # 변수 정리하기
    returncode = running_subprocess.poll()
    running_subprocess = None

    # 명령어 실행에 실패했다면 빨간 글씨로 표시하기
    styles = {'color': 'green'}
    if returncode != 0:
        styles['color'] = 'red'

    html_logger.value += '</div>'
    log(f"=> {returncode}", styles=styles)

    # 반환 코드가 정상이 아니라면 예외 발생하기
    if returncode != 0 and throw:
        raise Exception(f'프로세스가 {returncode} 코드를 반환했습니다')

    return returncode


# ==============================
# 작업 경로
# ==============================
path_to = {
    'packages': '/content/packages',
    'repository': '/content/repository'
}

def update_path_to(path_to_workspace: str) -> None:
    log(f'작업 공간 경로를 "{path_to_workspace}" 으로 변경했습니다')

    path_to['workspace'] = path_to_workspace
    path_to['outputs'] = f"{path_to['workspace']}/outputs"
    path_to['models'] = f"{path_to['workspace']}/models"
    path_to['embeddings'] = f"{path_to['workspace']}/embeddings"
    path_to['ui_config_file'] = f"{path_to['workspace']}/ui-config.json"
    path_to['ui_settings_file'] = f"{path_to['workspace']}/config.json"

    os.makedirs(path_to['workspace'], exist_ok=True)
    os.makedirs(path_to['packages'], exist_ok=True)
    os.makedirs(path_to['embeddings'], exist_ok=True)


# ==============================
# 사용자 설정
# ==============================
CHECKPOINTS = {
    # 'Standard Model 1.4': {
    #   'files': [{'url': 'https://public.vmm.pw/aeon/models/sd-v1-4.ckpt'}]
    # },

    # NAI leaks
    'NAI - animefull-final-pruned': {
        'files': [
            {
                'url': 'https://anonfiles.com/n6h3Q0Bdyf',
                'args': ['-o', 'nai-animefull-final-pruned.ckpt']
            },
            {
                'url': 'https://anonfiles.com/66c1QcB7y6',
                'args': ['-o', 'nai-animefull-final-pruned.vae.pt']
            },
            {
                'url': 'https://gist.githubusercontent.com/toriato/ae1f587f4d1e9ee5d0e910c627277930/raw/6019f8782875497f6e5b3e537e30a75df5b64812/animefull-final-pruned.yaml',
                'args': ['-o', 'nai-animefull-final-pruned.yaml']
            }
        ]
    },
    'NAI - animefull-latest': {
        'files': [
            {
                'url': 'https://anonfiles.com/8fm7QdB1y9',
                'args': ['-o', 'nai-animefull-latest.ckpt']
            },
            {
                'url': 'https://anonfiles.com/66c1QcB7y6',
                'args': ['-o', 'nai-animefull-latest.vae.pt']
            },
            {
                'url': 'https://gist.githubusercontent.com/toriato/ae1f587f4d1e9ee5d0e910c627277930/raw/6019f8782875497f6e5b3e537e30a75df5b64812/animefull-latest.yaml',
                'args': ['-o', 'nai-animefull-latest.yaml']
            }
        ]
    },

    # Waifu stuffs
    # 'Waifu Diffusion 1.2': {
    #   'files': [{'url': 'https://public.vmm.pw/aeon/models/wd-v1-2-full-ema-pruned.ckpt'}]
    # },
    'Waifu Diffusion 1.3': {
        'files': [{
            'url': 'https://huggingface.co/hakurei/waifu-diffusion-v1-3/resolve/main/wd-v1-3-float16.ckpt',
            'args': ['-o', 'wd-v1-3-epoch09-float16.ckpt']
        }]
    },

    # Trinart2
    'Trinart Stable Diffusion v2 60,000 Steps': {
        'files': [{'url': 'https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step60000.ckpt'}]
    },
    'Trinart Stable Diffusion v2 95,000 Steps': {
        'files': [{'url': 'https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step95000.ckpt'}]
    },
    'Trinart Stable Diffusion v2 115,000 Steps': {
        'files': [{'url': 'https://huggingface.co/naclbit/trinart_stable_diffusion_v2/resolve/main/trinart2_step115000.ckpt'}]
    },

    # Kinky c:
    # 'gg1342_testrun1': {
    #   'files': [{'url': 'https://public.vmm.pw/aeon/models/gg1342_testrun1_pruned.ckpt'}]
    # },
    # 'Hentai Diffusion RD1412': {
    #   'files': [{
    #     'url': 'https://public.vmm.pw/aeon/models/RD1412-pruned-fp16.ckpt',
    #     'args': ['-o', 'hentai_diffusion-rd1412-pruned-fp32.ckpt']
    #   }]
    # },
    # 'Bare Feet / Full Body b4_t16_noadd': {
    #   'files': [{'url': 'https://public.vmm.pw/aeon/models/bf_fb_v3_t4_b16_noadd-ema-pruned-fp16.ckpt'}]
    # },
    # 'Lewd Diffusion 70k (epoch 2)': {
    #   'files': [{'url': 'https://public.vmm.pw/aeon/models/LD-70k-2e-pruned.ckpt'}]
    # },

    # More kinky c:<
    # 'Yiffy (epoch 18)': {
    #   'files': [{'url': 'https://public.vmm.pw/aeon/models/yiffy-e18.ckpt'}]
    # },
    'Furry (epoch 4)': {
        'files': [{'url': 'https://iwiftp.yerf.org/Furry/Software/Stable%20Diffusion%20Furry%20Finetune%20Models/Finetune%20models/furry_epoch4.ckpt'}]
    },
    'Zack3D Kinky v1': {
        'files': [{'url': 'https://iwiftp.yerf.org/Furry/Software/Stable%20Diffusion%20Furry%20Finetune%20Models/Finetune%20models/Zack3D_Kinky-v1.ckpt'}]
    },
    # 'R34 (epoch 1)': {
    #   'files': [{'url': 'https://public.vmm.pw/aeon/models/r34_e1.ckpt'}]
    # },
    # 'Pony Diffusion': {
    #  'files': [{ 'url': 'https://public.vmm.pw/aeon/models/pony_sfw_80k_safe_and_suggestive_500rating_plus-pruned.ckpt'}]
    # },
    'Pokemon': {
        'files': [{
            'url': 'https://huggingface.co/justinpinkney/pokemon-stable-diffusion/resolve/main/ema-only-epoch%3D000142.ckpt',
            'args': ['-o', 'pokemon-ema-pruned.ckpt']
        }]
    },

    # Others...
    'Dreambooth - Hiten': {
        'files': [{'url': 'https://huggingface.co/BumblingOrange/Hiten/resolve/main/Hiten%20girl_anime_8k_wallpaper_4k.ckpt'}]
    },
}

# @markdown ### ***체크포인트 모델 선택***
# @markdown - [모델 별 설명 및 다운로드 주소](https://rentry.org/sdmodels)
CHECKPOINT = 'NAI - animefull-final-pruned' # @param ['NAI - animefull-final-pruned', 'NAI - animefull-latest', 'Waifu Diffusion 1.3', 'Trinart Stable Diffusion v2 60,000 Steps', 'Trinart Stable Diffusion v2 95,000 Steps', 'Trinart Stable Diffusion v2 115,000 Steps', 'Furry (epoch 4)', 'Zack3D Kinky v1', 'Pokemon', 'Dreambooth - Hiten'] {allow-input: true}

# @markdown ### ***구글 드라이브 동기화를 사용할건지?***
USE_GOOGLE_DRIVE = True  # @param {type:"boolean"}

# @markdown ### ***구글 드라이브 경로***
PATH_TO_GOOGLE_DRIVE = 'SD'  # @param {type:"string"}

# @markdown ### ***xformers 를 사용할지?***
# @markdown 켜두면 10-15% 정도의 성능 향상을 ***보일 수도 있음***
# @markdown <br>패키지를 직접 빌드하는데 30분에서 한 시간 정도 걸림
# @markdown <br>구글 드라이브 동기화 아직 구현 안해둬서 개인 컴퓨터에서만 사용하는 걸 추천함
USE_XFORMERS = False  # @param {type:"boolean"}

# @markdown ### ***DeepDanbooru 를 사용할지?***
USE_DEEPDANBOORU = True  # @param {type:"boolean"}

# 현재 코랩 환경에서 구동 중인지?
IN_COLAB = find_spec('google.colab') is not None


# ==============================
# 패키지 준비
# ==============================
def prepare_aria2() -> None:
    log('aria2 패키지를 설치합니다')
    execute_subprocess(
        ['apt', 'install', '-y', 'aria2'])

    # 설정 파일 만들기
    log('aria2 설정 파일을 만듭니다')
    os.makedirs(os.path.join(Path.home(), '.aria2'), exist_ok=True)
    with open(Path.joinpath(Path.home(), '.aria2', 'aria2.conf'), "w") as f:
        f.write("""
summary-interval=10
allow-overwrite=true
always-resume=true
disk-cache=64M
continue=true
min-split-size=8M
max-concurrent-downloads=8
max-connection-per-server=8
max-overall-download-limit=0
max-download-limit=0
split=8
seed-time=0
""")


# ==============================
# 구글 드라이브 동기화
# ==============================
def mount_google_drive() -> None:
    log('구글 드라이브 마운트를 시도합니다')

    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

    # 전체 경로 업데이트
    update_path_to(os.path.join('/content/drive/MyDrive', PATH_TO_GOOGLE_DRIVE))


# ==============================
# 파일 다운로드
# ==============================
def download(url: str, args=[]):
    # anonfile CDN 주소 가져오기
    if url.startswith('https://anonfiles.com/'):
        matches = re.search('https://cdn-[^\"]+', requests.get(url).text)
        if not matches:
            raise Exception('anonfiles 에서 CDN 주소를 파싱하는데 실패했습니다')

        url = matches[0]

    # Aria2 로 모델 받기
    log(f"파일 다운로드를 시도합니다: {url}")
    execute_subprocess(['aria2c', *args, url])
    log('파일을 성공적으로 받았습니다!')


def download_checkpoint(checkpoint: str) -> None:
    try:
        prepare_aria2()

        # 선택한 체크포인트 정보 가져오기
        if checkpoint in CHECKPOINTS:
            checkpoint = CHECKPOINTS[checkpoint]
        else:
            # 미리 선언된 체크포인트가 아니라면 주소로써 사용하기
            checkpoint = {'files': [{'url': checkpoint}]}

        # Aria2 로 모델 받기
        # TODO: 토렌트 마그넷 주소 지원
        log(f"파일 {len(checkpoint['files'])}개를 받습니다")

        for f in checkpoint['files']:
            file = json.loads(json.dumps(f))

            if 'args' not in file:
                file['args'] = []

            # 모델 받을 기본 디렉터리 경로 잡아주기
            if '-d' not in file['args']:
                file['args'] = [
                    '-d', f"{path_to['models']}/Stable-diffusion", *file['args']]

            download(**file)

    finally:
        is_fetching_checkpoint = False


def has_checkpoint() -> bool:
    for p in Path(f"{path_to['models']}/Stable-diffusion").glob('**/*.ckpt'):
        # aria2 로 받다만 파일은 무시하기
        if os.path.isfile(f'{p}.aria2'):
            continue

        return True
    return False


# ==============================
# 파이썬 패키지
# ==============================
def install_python_package(pkgs: List[str], args=['-I', '--progress-bar=off', '--prefer-binary'],
                           skip_if_has_package:List[str]=None, persist=False) -> None:
    # 존재한다면 스킵할 패키지가 존재하는지 확인하기
    if skip_if_has_package and len(filter_installed_python_packages([skip_if_has_package])) < 1:
        log(f'{pkgs} 패키지가 이미 존재합니다, 설치를 넘깁니다')
        return

    # 영구 유지할 패키지는 외부 패키지 디렉터리에 저장하기
    if persist:
        args = [*args, f"--target={path_to['packages']}"]

    log(f'{pkgs} 패키지가 존재하지 않습니다, 설치를 시도합니다')

    # execute_subprocess(['python', '-m', 'pip', 'install', '--upgrade', 'setuptools'])
    execute_subprocess(['python', '-m', 'pip', 'install', *args, *pkgs])


def installed_python_packages() -> List[str]:
    pkgs = !python -m pip list | tail -n+3 | cut -d' ' -f1
    return pkgs


def filter_installed_python_packages(pkgs: List[str]) -> List[str]:
    installed_pkgs = installed_python_packages()
    return [
        p for p in pkgs if
        # 빈 줄 제외
        not p == ''

        # 주석 처리된 패키지 제외
        and not p.startswith('#')

        # 설치된 패키지 제외
        and re.search('[a-zA-Z0-9-_]+', p)[0].replace('_', '-') not in installed_pkgs
    ]


# ==============================
# WebUI 레포지토리 및 종속 패키지 설치
# ==============================
def patch_webui_repository() -> None:
    # 모델 용량이 너무 커서 코랩 메모리 할당량을 초과하면 프로세스를 강제로 초기화됨
    # 이를 해결하기 위해선 모델 맵핑 위치를 VRAM으로 변경해줘야함
    # Thanks to https://gist.github.com/td2sk/e32a39344537fb3cd756ef4abdd3d371
    # TODO: 코랩에서만 발생하는 문제인지?
    log('WebUI 패치 -> 모델 맵핑 위치를 변경합니다')
    execute_subprocess([
        'sed',
        '-i',
        '''s/map_location="cpu"/map_location=torch.device("cuda")/g''',
        f"{path_to['repository']}/modules/sd_models.py"
    ])


def install_webui_dependencies() -> None:
    log('WebUI 종속 패키지를 설치합니다')

    # 코랩에선 필요 없으나 다른 환경에선 높은 확률로 설치 필요한 패키지들
    execute_subprocess(
        ['apt', 'install', '-y', 'build-essential', 'libgl1'])

    # 설치된 PyTorch 가 최신 GPU 지원하지 않을 수 있기 때문에 최신 버전으로 받아주기
    # install_python_package(
    #     ['torch', 'torchvision'],
    #     [
    #         '--ignore-installed',
    #         '--extra-index-url=https://download.pytorch.org/whl/cu116'
    #     ],
    #     skip_if_has_package='torch',
    #     persist=True
    # )

    # xformers 패키지 설치하기
    # https://github.com/AUTOMATIC1111/stable-diffusion-webui/wiki/Xformers
    # if USE_XFORMERS and 'xformers' not in installed_python_packages():
    #     cwd = f"{path_to['repository']}/repositories/xformers"

    #     log('xformers 패키지를 수동으로 설치합니다, 레포지토리를 가져옵니다')
    #     rmtree('xformers', ignore_errors=True)
    #     execute_subprocess(['git', 'clone', 'https://github.com/facebookresearch/xformers.git', cwd])
    #     execute_subprocess(['git', 'submodule', 'update', '--init', '--recursive'], cwd=cwd)
    #     execute_subprocess(['python', '-m', 'pip', 'install', '-r', 'requirements.txt'], cwd=cwd)

    #     log('xformers 패키지를 빌드하고 설치합니다')
    #     execute_subprocess(['python', '-m', 'pip', 'install', 'setuptools==49.6.0'])
    #     execute_subprocess(['python', 'setup.py', 'install'], cwd=cwd)


def setup_webui() -> None:
    need_clone = True

    # 이미 디렉터리가 존재한다면 정상적인 레포인지 확인하기
    if os.path.isdir(path_to['repository']):
        try:
            log('WebUI 레포지토리를 풀(업데이트) 합니다')

            # 사용자 파일만 남겨두고 레포지토리 초기화하기
            # https://stackoverflow.com/a/12096327
            execute_subprocess(['git', 'add', '-f', 'scripts', 'repositories'], cwd=path_to['repository'])
            execute_subprocess(['git', 'checkout', '.'], cwd=path_to['repository'])
            execute_subprocess(['git', 'pull'], cwd=path_to['repository'])
            patch_webui_repository()

        except:
            log('레포지토리가 잘못됐습니다, 디렉터리를 제거합니다')

    if need_clone:
        log('WebUI 레포지토리를 클론합니다')
        rmtree(path_to['repository'], ignore_errors=True)
        execute_subprocess(['git', 'clone', 'https://github.com/AUTOMATIC1111/stable-diffusion-webui', path_to['repository']])

    install_webui_dependencies()


def parse_webui_output(out: str) -> bool:
    matches = re.search('https://\d+\.gradio\.app', out)
    if matches:
        log('******************************************', styles={'color':'green'})
        log(f'성공적으로 웹UI를 실행했습니다, 아래 주소에 접속해주세요!\n{matches[0]}',
            styles={
                'background-color': 'green',
                'font-weight': 'bold',
                'font-size': '1.5em',
                'line-height': '1.5em',
                'color': 'black'
            })
        log('******************************************', styles={'color':'green'})

        dialog(f'''
        <p>성공적으로 웹UI를 실행했습니다!</p>
        <p><a target="_blank" href="{matches[0]}">{matches[0]}</a></p>
        ''')


def start_webui(args: List[str]=[], env={}) -> None:
    global running_subprocess

    if running_subprocess is not None:
        if 'launch.py' in running_subprocess.args:
            log('이미 실행 중인 웹UI를 종료하고 다시 시작합니다')
            running_subprocess.kill()
            running_subprocess = None

        raise ('이미 다른 프로세스가 실행 중입니다, 잠시 후에 실행해주세요')

    execute_subprocess(
        ['python', 'launch.py', *args],
        parser=parse_webui_output,
        cwd=path_to['repository'],
        env={
            **os.environ,
            'PYTHONUNBUFFERED': '1',
            'REQS_FILE': 'requirements.txt',
            **env
        }
    )


# ==============================
# 보고서
# ==============================
def generate_report() -> str:
    import traceback
    from distutils.spawn import find_executable

    packages = !pip freeze
    packages_310 = !python -m pip freeze

    ex_type, ex_value, ex_traceback = sys.exc_info()
    traces = map(lambda v: f'{v[0]}#{v[1]}\n\t{v[2]}\n\t{v[3]}', traceback.extract_tb(ex_traceback))

    def format_list(value):
        if isinstance(value, dict):
            return '\n'.join(map(lambda kv: f'{kv[0]}: {kv[1]}', value.items()))
        else:
            return '\n'.join(value)

    payload = f"""
{html_logger.raw}
# {ex_type.__name__}: {ex_value}
{format_list(traces)}

# platform
{platform.platform()}

# options
CHECKPOINT: {CHECKPOINT}
USE_GOOGLE_DRIVE: {USE_GOOGLE_DRIVE}
PATH_TO_GOOGLE_DRIVE: {PATH_TO_GOOGLE_DRIVE}
USE_XFORMERS: {USE_XFORMERS}
USE_DEEPDANBOORU: {USE_DEEPDANBOORU}

# paths
{format_list(path_to)}

# models
{format_list(glob.glob(f"{path_to['models']}/**/*"))}

# {sys.executable}
{format_list(packages)}

# {find_executable('python')}
{format_list(packages_310)}
"""

    res = requests.post('https://hastebin.com/documents',
                        data=payload.encode('utf-8'))

    return f"https://hastebin.com/{json.loads(res.text)['key']}"


# ==============================
# 자 드게제~
# ==============================
try:
    # 코랩 폼 입력 란을 생성을 위한 코드
    # log(', '.join(map(lambda s:f"'{s}'", CHECKPOINTS.keys())))

    # 기본 작업 경로 설정
    update_path_to(os.curdir)

    if IN_COLAB:
        log('코랩 환경이 감지됐습니다')

        import torch
        assert torch.cuda.is_available(), 'GPU 가 없습니다, 런타임 유형이 잘못됐거나 GPU 할당량이 초과된 것 같습니다'

        # 코랩 환경이라면 /content 디렉터리는 스토리지 속도가 느리기 때문에
        # /usr/local 속에서 구동할 필요가 있음
        log('레포지토리 디렉터리를 "/usr/local/repository" 로 변경합니다')
        path_to['repository'] = '/usr/local/repository'

        # 구글 드라이브 마운팅 시도
        if USE_GOOGLE_DRIVE:
            mount_google_drive()

    # 인터페이스 출력
    btn_download_checkpoint = widgets.Button(description='체크포인트 받기')
    btn_download_checkpoint.on_click(
        lambda _: download_checkpoint(CHECKPOINT)
    )

    display(
        widgets.VBox([
            btn_download_checkpoint,
            html_dialog,
            html_logger
        ])
    )

    # 구동 필수 패키지 준비
    prepare_aria2()

    # 이후 아카이브 전 변경 여부 확인을 위해 현재 패키지 목록 받아두기
    packages = Counter(installed_python_packages())

    # 체크포인트가 없을 시 다운로드
    if not has_checkpoint():
        download_checkpoint(CHECKPOINT)

    # WebUI 가져오기
    setup_webui()

    # WebUI 기본 설정 적용
    with open(path_to['ui_config_file'], 'w+') as file:
        configs = {
            # NAI 기본 설정(?)
            'txt2img/Prompt/value': 'best quality, masterpiece',
            'txt2img/Negative prompt/value': 'lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry',
            'txt2img/Sampling Steps/value': 28,
            'txt2img/Width/value': 512,
            'txt2img/Height/value': 768,
            'txt2img/CFG Scale/value': 12,
        }

        # 기존 설정 우선으로 사용하기
        try:
            configs = { **configs, **json.loads(file.read()) }
        except ValueError:
            pass

        file.write(json.dumps(configs, indent=4))

    with open(path_to['ui_settings_file'], 'w+') as file:
        configs = {
            'save_txt': True,

            # 결과 이미지 디렉터리
            'outdir_txt2img_samples': f"{path_to['outputs']}/txt2img-samples",
            'outdir_img2img_samples': f"{path_to['outputs']}/img2img-samples",
            'outdir_extras_samples': f"{path_to['outputs']}/extras-samples",
            'outdir_txt2img_grids': f"{path_to['outputs']}/txt2img-grids",
            'outdir_img2img_grids': f"{path_to['outputs']}/img2img-grids",

            # NAI 기본 설정(?)
            'eta_ancestral': 0.2,
            'CLIP_stop_at_last_layers': 2,
        }

        # 기존 설정 우선으로 사용하기
        try:
            configs = { **configs, **json.loads(file.read()) }
        except ValueError:
            pass

        file.write(json.dumps(configs, indent=4))

    # WebUI 실행
    args = [
        '--share',
        '--gradio-debug',

        # 동적 경로들
        f"--ckpt-dir={path_to['models']}/Stable-diffusion",
        f"--embeddings-dir={path_to['embeddings']}",
        f"--hypernetwork-dir={path_to['models']}/hypernetworks",
        f"--codeformer-models-path={path_to['models']}/Codeformer",
        f"--gfpgan-models-path={path_to['models']}/GFPGAN",
        f"--esrgan-models-path={path_to['models']}/ESRGAN",
        f"--bsrgan-models-path={path_to['models']}/BSRGAN",
        f"--realesrgan-models-path={path_to['models']}/RealESRGAN",
        f"--scunet-models-path={path_to['models']}/ScuNET",
        f"--swinir-models-path={path_to['models']}/SwinIR",
        f"--ldsr-models-path={path_to['models']}/LDSR",

        f"--ui-config-file={path_to['ui_config_file']}",
        f"--ui-settings-file={path_to['ui_settings_file']}",
    ]

    cmd_args = [ '--skip-torch-cuda-test' ]

    if USE_XFORMERS:
        cmd_args = [*cmd_args, '--xformers']

    if USE_DEEPDANBOORU:
        cmd_args = [*cmd_args, '--deepdanbooru']

    start_webui(args, env={'COMMANDLINE_ARGS': ' '.join(cmd_args)})

# ^c 종료 무시하기
except KeyboardInterrupt:
    pass

# 오류 발생하면 보고서 생성하고 표시하기
except:
    _, ex_value, _ = sys.exc_info()
    report_url = generate_report()

    log('******************************************', styles={'color':'red'})
    log(f'오류가 발생했습니다, 아래 주소를 복사해 보고해주세요!\n{report_url}',
        styles={
            'background-color': 'red',
            'font-weight': 'bold',
            'font-size': '1.5em',
            'line-height': '1.5em',
            'color': 'black'
        })
    log('******************************************', styles={'color':'red'})
    log(f'{ex_value}', styles={'color':'red'})

    dialog(
        f'''
        <p>오류가 발생했습니다, 아래 주소를 복사해 보고해주세요!</p>
        <p><strong>{generate_report()}</strong></p>
        ''',
        preset='error'
    )
